In [19]:
import pandas as pd
import ast
import re
df_movies = pd.read_parquet('datasets/clean_movies_dataset.parquet')
df_credits = pd.read_parquet('datasets/credits.parquet')

In [20]:
df_credits.cast.apply(lambda x : type(x))

0        <class 'str'>
1        <class 'str'>
2        <class 'str'>
3        <class 'str'>
4        <class 'str'>
             ...      
45471    <class 'str'>
45472    <class 'str'>
45473    <class 'str'>
45474    <class 'str'>
45475    <class 'str'>
Name: cast, Length: 45476, dtype: object

In [21]:
df_movies.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'release_year', 'return'],
      dtype='object')

### def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en el mes de X

In [41]:
def cantidad_filmaciones_mes(mes:str):
    '''Se ingresa el mes y la funcion retorna la cantidad de peliculas que se estrenaron ese mes historicamente'''
    mes = mes.lower()
    df_movies.release_date = pd.to_datetime(df_movies.release_date)
    def mes_a_numero (mes):
        meses = {
            'enero':1,'febrero':2,'marzo':3,'abril':4,
            'mayo':5,'junio':6,'julio':7,'agosto':8,
            'septiembre':9,'octubre':10,'noviembre':11,'diciembre':12
            }
        return meses[mes]
    cantidad = df_movies.release_date.loc[df_movies.release_date.dt.month == mes_a_numero(mes)].count().item()
    return {'mes':mes, 'cantidad':cantidad}

- prueba

In [42]:
cantidad_filmaciones_mes('febrero')

{'mes': 'febrero', 'cantidad': 3032}

### def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X

In [43]:
def cantidad_filmaciones_dia(dia:str):
    '''Se ingresa el dia y la funcion retorna la cantidad de peliculas que se estrebaron ese dia historicamente'''
    dia = dia.lower()
    df_movies.release_date = pd.to_datetime(df_movies.release_date, errors= 'coerce')
    def dia_numero(dia_numero):
        dias = {'lunes' : 0,'martes' : 1, 'miercoles': 2, 'jueves':3, 'viernes':4, 'sabado':5, 'domingo':6}
        return dias[dia_numero]
    cantidad = df_movies.release_date.loc[df_movies.release_date.dt.weekday == dia_numero(dia)].count()
    return {'dia' : dia , 'cantidad' : cantidad.item()}

- prueba

In [44]:
cantidad_filmaciones_dia('lunes')

{'dia': 'lunes', 'cantidad': 3503}

### def score_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.
                    Ejemplo de retorno: La película X fue estrenada en el año X con un score/popularidad de

In [47]:
def score_titulo(titulo_de_la_filmacion: str):
    titulo_de_la_filmacion = titulo_de_la_filmacion.lower()
    respuesta = df_movies[['title','release_year','popularity']].loc[df_movies.title.apply(lambda x : x.lower()) == titulo_de_la_filmacion]
    titulo = str(respuesta['title'].values[0])
    anio = int(respuesta['release_year'].values[0])
    popularidad = float(respuesta['popularity'].values[0])
    return {'titulo':titulo, 'anio':anio, 'popularidad':popularidad}

- prueba

In [48]:
score_titulo('Toy Story')

{'titulo': 'Toy Story', 'anio': 1995, 'popularidad': 21.946943}

### def votos_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.
                    Ejemplo de retorno: La película X fue estrenada en el año X. La misma cuenta con un total de X valoraciones, con un promedio de X

In [53]:

def votos_titulo(titulo:str):
    '''Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. 
    La misma variable deberá de contar con al menos 2000 valoraciones, 
    caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.'''
    respuesta = df_movies[['title','release_year','vote_count','vote_average']].loc[df_movies.title.apply(lambda x : x.lower()) == titulo.lower()]
    titulo = str(respuesta['title'].values[0])
    anio = int(respuesta['release_year'].values[0])
    voto_total = int(respuesta['vote_count'].values[0])
    voto_promedio = float(respuesta['vote_average'].values[0])
    if respuesta['vote_count'].values[0] <2000:
        return 'La variable ingresada no cumple con la condición necesaria de tener más de 2000 valoraciones'
    else:
        return {'titulo':titulo, 'anio':anio, 'voto_total':voto_total, 'voto_promedio':voto_promedio}

- prueba

In [54]:
votos_titulo('Toy Story')

{'titulo': 'Toy Story', 'anio': 1995, 'voto_total': 5415, 'voto_promedio': 7.7}

### def get_actor( nombre_actor ): Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.
                    Ejemplo de retorno: El actor X ha participado de X cantidad de filmaciones, el mismo ha conseguido un retorno de X con un promedio de X por filmación

In [58]:
def get_actor(nombre_actor:str):
    '''Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. 
    Además, la cantidad de películas que en las que ha participado y el promedio de retorno'''
    resultado = df_credits[df_credits['cast'].apply(lambda x : nombre_actor.lower().rstrip() in x.lower())]
    cantidad_filmaciones = resultado.shape[0]
    lista_proyectos = resultado.id.tolist()
    proyectos = df_movies[df_movies['id'].isin(lista_proyectos)]
    retorno_total = sum(proyectos.revenue.tolist())
    retorno_promedio = round(retorno_total/cantidad_filmaciones)
    return {'actor':nombre_actor, 'cantidad_filmaciones':cantidad_filmaciones, 'retorno_total':retorno_total, 'retorno_promedio':retorno_promedio}

- prueba

In [59]:
get_actor('Robin Williams')

{'actor': 'Robin Williams',
 'cantidad_filmaciones': 83,
 'retorno_total': 6356609847,
 'retorno_promedio': 76585661}

### def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.

In [60]:
def get_director(nombre_director:str):
    ''' Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. 
    Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.'''
    condicion1 = df_credits['crew'].str.contains(r'\b{}\b'.format(nombre_director.title()), regex=True)
    condicion2 = df_credits['crew'].str.contains(r'\b{}\b'.format('Director'), regex=True)
    resultado = df_credits[condicion1 & condicion2]

    lista_proyectos = resultado.id.tolist()
    proyectos = df_movies[df_movies['id'].isin(lista_proyectos)]

    revenue_pelicula = proyectos.revenue.tolist()
    retorno_pelicula = proyectos['return'].tolist()
    budget_pelicula = proyectos.budget.tolist()
    retorno_total_director = sum(revenue_pelicula)
    peliculas = proyectos.title.tolist()
    anio_peliculas = [fecha.strftime('%Y-%m-%d') for fecha in proyectos.release_date.dt.date]
    
    return {
        'director':nombre_director, 
        'retorno_total_director':retorno_total_director, 
        'peliculas':peliculas, 
        'anio':anio_peliculas, 
        'retorno_pelicula':retorno_pelicula, 
        'budget_pelicula':budget_pelicula, 
        'revenue_pelicula':revenue_pelicula}
        

- prueba

In [61]:
get_director('Joe Johnston')

{'director': 'Joe Johnston',
 'retorno_total_director': 1531004802,
 'peliculas': ['Jumanji',
  'The Pagemaster',
  'Honey, I Shrunk the Kids',
  'The Rocketeer',
  'Willow',
  'October Sky',
  'Jurassic Park III',
  'Hidalgo',
  'Batteries not Included',
  'The Ewok Adventure',
  'The Wolfman',
  'Captain America: The First Avenger',
  'Not Safe for Work'],
 'anio': ['1995-12-15',
  '1994-11-23',
  '1989-06-22',
  '1991-06-21',
  '1988-05-01',
  '1999-02-19',
  '2001-07-18',
  '2004-03-05',
  '1987-12-18',
  '1985-02-13',
  '2010-02-11',
  '2011-07-22',
  '2014-05-09'],
 'retorno_pelicula': [4.0430346,
  0.5063217777777778,
  6.960130375,
  1.4761904761904763,
  1.6362818,
  0.0,
  3.9653850430107527,
  1.0810345,
  0.0,
  0.0,
  0.0,
  2.646926957142857,
  0.0],
 'budget_pelicula': [65000000,
  27000000,
  32000000,
  42000000,
  35000000,
  0,
  93000000,
  100000000,
  0,
  3000000,
  150000000,
  140000000,
  0],
 'revenue_pelicula': [262797249,
  13670688,
  222724172,
  62000000


# ML
@app.get('/recomendacion/{titulo}')
def recomendacion(titulo:str):
    '''Ingresas un nombre de pelicula y te recomienda las similares en una lista'''
    return {'lista recomendada': respuesta}